In [12]:
import keras
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
import mlflow
from mlflow.models import infer_signature

In [13]:
data=pd.read_csv('winequality-white.csv',sep=';')
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [14]:
train,test=train_test_split(data,test_size=0.20,random_state=33)


In [15]:
x_train=train.drop(['quality'],axis=1).values
y_train=train[['quality']].values.ravel()
x_test=test.drop(['quality'],axis=1).values
y_test=test[['quality']].values.ravel()

In [16]:
x_train,x_valid,y_train,y_valid=train_test_split(x_train,y_train,test_size=0.20,random_state=33)
signature=infer_signature(x_train,y_train)

In [17]:
y_train

array([6, 6, 7, ..., 6, 6, 6])

In [18]:
##ann model

def train_model(params,epochs,x_train,y_train,x_valid,y_valid,x_test,y_test):
    mean=np.mean(x_train,axis=0)
    var=np.var(x_train,axis=0)
    
    model=keras.Sequential(
        [
            keras.Input([x_train.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)
        ]
    )
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )
    
    with mlflow.start_run(nested=True):
        model.fit(x_train,y_train,validation_data=(x_valid,y_valid),
                  epochs=epochs,
                  batch_size=64)
        eval_result=model.evaluate(x_valid,y_valid,batch_size=64)
        eval_rmse=eval_result[1]
        
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)
        
        mlflow.tensorflow.log_model(model,"model",signature=signature)
        return {"loss":eval_rmse,"status":STATUS_OK,"model":model}
    
        

In [19]:
def objective(params):
    result=train_model(
        params,
        epochs=3,
        x_train=x_train,
        y_train=y_train,
        x_valid=x_valid,
        y_valid=y_valid,
        x_test=x_test,
        y_test=y_test,
    )
    return result

In [20]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [22]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


2025/03/19 14:48:09 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 22s 477ms/step - loss: 33.8290 - root_mean_squared_error: 5.8163
31/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 23.2675 - root_mean_squared_error: 4.7918   
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 19.3537 - root_mean_squared_error: 4.3378 - val_loss: 2.6812 - val_root_mean_squared_error: 1.6374

Epoch 2/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 2.5305 - root_mean_squared_error: 1.5908
47/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.3433 - root_mean_squared_error: 1.5300 
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.3349 - root_mean_squared_error: 1.5272 - val_loss: 1.7497 - val_root_mean_squared_error: 1.3228

Epoch 3/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 1.5422 - root_mean_squared_error: 1.2418
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.8333 - root_mean_squared_error: 1.3534 - v

In [23]:
from mlflow.models import validate_serving_input

model_uri = 'runs:/8897403be189454e84031ab27eef8e1e/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(x_test)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

c:\Users\kk061\OneDrive\Desktop\python\Mlflow Main\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


array([[5.518791 ],
       [5.717313 ],
       [5.818749 ],
       [5.1180954],
       [5.8728757],
       [5.819234 ],
       [5.832044 ],
       [6.380312 ],
       [6.218156 ],
       [5.500543 ],
       [5.52557  ],
       [6.00157  ],
       [5.3348546],
       [4.902037 ],
       [5.108776 ],
       [5.1266217],
       [6.6348815],
       [6.088015 ],
       [5.097349 ],
       [5.660017 ],
       [5.932527 ],
       [6.6329994],
       [5.92798  ],
       [5.3636403],
       [5.55739  ],
       [5.6548786],
       [6.470122 ],
       [5.8646917],
       [5.556592 ],
       [4.261587 ],
       [5.656905 ],
       [6.1703258],
       [4.3461447],
       [6.042709 ],
       [6.164338 ],
       [5.4321547],
       [5.2465286],
       [5.6237   ],
       [5.7533703],
       [5.319595 ],
       [5.1233597],
       [5.5093117],
       [6.1333795],
       [4.9182634],
       [5.553092 ],
       [6.293436 ],
       [5.987589 ],
       [6.230937 ],
       [6.0601397],
       [5.848446 ],


In [24]:
mlflow.register_model(model_uri,"wine-quality")

Successfully registered model 'wine-quality'.
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1742376210836, current_stage='None', description=None, last_updated_timestamp=1742376210836, name='wine-quality', run_id='8897403be189454e84031ab27eef8e1e', run_link=None, source='file:///c:/Users/kk061/OneDrive/Desktop/python/Mlflow%20Main/3-deep%20learning%20project/mlruns/353409921340440663/8897403be189454e84031ab27eef8e1e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>